In [1]:
import nltk
import numpy as np
import pandas as pd
import os
import unicodedata

import google_conf

In [52]:
# uncomment to establish a connection with google sheet
# pop_data = google_conf.setup(sheet_url="https://docs.google.com/spreadsheets/d/1o8L0Rt4CzQ_DcYdaKgCdige4nJs0sydlTGiMBSstQtc/edit?usp=sharing", service_account_path="../../ServiceAccountsKey.json")


In [171]:
articles_metadata = pd.read_csv("../data/articles_metadata_INPUT.csv")
articles_metadata.head(5)

,year,authors,journal,title,type,pdf?,keywords,keywords_online?,notes
0,2011,Bos et al.,Political Communication,How the Media Shape Perceptions of Right-Wing ...,article,y,y,y,NaN
1,2019,Blassnig et al.,Political Communication,Hitting a Nerve: Populist News Articles Lead t...,article,y,y,y,NaN
2,2017,Caramani,American Political Science Review,Will vs. Reason: The Populist and Technocratic...,article,y,y,NaN,NaN
3,2020,"Wuttke,Schimpf, Schoen",American Political Science Review,When the Whole Is Greater than the Sum of Its ...,article,y,y,NaN,NaN
4,2014,Treib,Journal of European Public Policy,"The voter says no, but nobody listens: causes ...",article,y,y,NaN,NaN


In [172]:
len(articles_metadata)

541

In [173]:
articles_metadata[~(articles_metadata["pdf?"]=="y")]

,year,authors,journal,title,type,pdf?,keywords,keywords_online?,notes
52,2018,Gamboa,Journal of Democracy,Latin America’s Shifting Politics: The Peace P...,article,n,y,y,NEJDE STÁHNOUT - NEMÁM PŘÍSTUP
461,2020,Schoenman,Communist and Post-Communist Studies,Old Wine in New Bottles? New Parties and Polic...,article,n,y,NaN,NEJDE STÁHNOUT - NEMÁM PŘÍSTUP


In [174]:
articles_metadata = articles_metadata[articles_metadata["pdf?"]=="y"].copy()
len(articles_metadata)

539

In [175]:
# Collect paths

In [176]:
# the corpus of PDF articles used within this project is under copy ritht. Therefore we cannot make this data publicly avaialable.

filepaths = []
for root, dir, files in os.walk("../data/large_data/articles"):
    journal = root.rpartition("/")[2]
    filepaths.extend([journal + "/" + fname for fname in files if ".pdf" in fname])
filepaths[:10]

['American Political Science Review/2017 - Caramani - Will vs. Reason_The Populist and Technocratic Forms of Political Representation and Their Critique to Party Government.pdf',
 'American Political Science Review/2020 - Wuttke, Schimpf, Schoen - When the Whole Is Greater than the Sum of Its Parts.pdf',
 'Political Research Quarterly/2020 - Castanho Silva et al. - An Empirical Comparison of Seven Populist Attitudes Scales.pdf',
 'Political Research Quarterly/2018 - Peel - The Populist Theory of the State in Early American Political Thought.pdf',
 'Political Research Quarterly/2020 - Kenny - “The Enemy of the People”_Populists and Press Freedom.pdf',
 'Political Research Quarterly/2018 - Fumurescu - The People’s Two Bodies_An Alternative Perspective on Populism and Elitism.pdf',
 'Political Research Quarterly/2016 - Spruyt et al. - Who Supports Populism and What Attracts People to It.pdf',
 'Political Research Quarterly/2018 - Love, Windsor - Populism and Popular Support_Vertical Accou

In [177]:
def get_file_path(row):
    # first start from title & author & year
    title_candidates = [fp for fp in filepaths if (row["title"].lower()[:11] in unicodedata.normalize("NFC", fp.lower())) & (row["authors"].lower() in unicodedata.normalize("NFC", fp.lower())) & (str(row["year"]) in fp.lower())]
    # start with title only
    #if len(title_candidates) == 0:
    #    title_candidates = [fp for fp in filepaths if row["title"].lower()[:15] in unicodedata.normalize("NFC", fp.lower())]
    #    if len(title_candidates) > 1:
    #        title_candidates = [fp for fp in title_candidates if row["authors"].lower() in unicodedata.normalize("NFC", fp.lower())]
    #        #title_candidates = [fp for fp in title_candidates if row["authors"].split(",")[0].lower() in unicodedata.normalize("NFC", fp.lower())]
    #    if len(title_candidates) > 1:
    #        title_candidates = [fp for fp in title_candidates if str(row["year"]) in fp.lower()]
    #    if len(title_candidates) > 1:
    #        title_candidates = [fp for fp in title_candidates if (row["journal"].lower()[:10]) in unicodedata.normalize("NFC", fp.lower())]
    # start again from author
    if len(title_candidates) == 0:
        title_candidates = [fp for fp in filepaths if row["authors"].split(",")[0].lower() in unicodedata.normalize("NFC", fp.lower())]
        if len(title_candidates) > 1:
            title_candidates = [fp for fp in title_candidates if str(row["year"]) in fp.lower()]
        if len(title_candidates) > 1:
            title_candidates = [fp for fp in title_candidates if (row["journal"].lower()[:10]) in unicodedata.normalize("NFC", fp.lower())]
        if len(title_candidates) > 1:
            title_candidates = [fp for fp in title_candidates if row["title"].lower()[:8] in unicodedata.normalize("NFC", fp.lower())]
        if len(title_candidates) > 1:
            title_candidates = [fp for fp in title_candidates if row["title"].lower()[-8:] in unicodedata.normalize("NFC", fp.lower())]
    # if still unsuccessful, then start with journal title
    if len(title_candidates) == 0:
        title_candidates = [fp for fp in filepaths if row["journal"].lower() in unicodedata.normalize("NFC", fp.lower())]
        if len(title_candidates) > 1:
            title_candidates = [fp for fp in title_candidates if str(row["year"]) in fp]
        if len(title_candidates) > 1:
            title_candidates = [fp for fp in title_candidates if row["title"].lower()[:6] in unicodedata.normalize("NFC", fp.lower())]
        if len(title_candidates) > 1:
            title_candidates = [fp for fp in title_candidates if row["authors"].lower()[:8] in unicodedata.normalize("NFC", fp.lower())]
    return title_candidates
    #if bool(title_candidates):
    #    return title_candidates[0]
    #else:
    #    return np.nan

articles_metadata["filepath"] = articles_metadata.apply(lambda row: get_file_path(row), axis=1)

In [178]:
(articles_metadata["filepath"].apply(len) ==0).sum()

0

In [186]:
articles_metadata[articles_metadata["authors"].str.contains("Breuilly")]

,pop_id,year,authors,journal,title,type,filepath
393,393,2017,Breuilly et al.,Nations and Nationalism,Editorial,editorial,Nations and Nationalism/2017 - Breuilly et al....
396,396,2019,Breuilly et al.,Nations and Nationalism,Editorial,editorial,Nations and Nationalism/2019 - Breuilly et al....


In [179]:
articles_metadata["filepath"] = articles_metadata["filepath"].apply(lambda x: x[0])

In [180]:
#export to googlesheets for visual inspection
#google_conf.set_with_dataframe(pop_data.add_worksheet("articles_metadata_paths", 1,1), articles_metadata)

In [181]:
missing = articles_metadata[articles_metadata["filepath"].apply(len) ==0]
missing

,year,authors,journal,title,type,pdf?,keywords,keywords_online?,notes,filepath


In [187]:
articles_metadata["pop_id"] = articles_metadata.index

In [188]:
articles_metadata = articles_metadata[['pop_id', 'year', 'authors', 'journal', 'title', 'type', 'filepath']]

In [189]:
articles_metadata.to_json("../data/article_metadata.json")
articles_metadata.to_csv("../data/article_metadata.csv")